<a href="https://colab.research.google.com/github/0323ali/Machine-Learning/blob/main/CNN_road_condition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pickle
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
a=cv2.imread("/content/drive/MyDrive/Data_set/Data/bump/Image00060.jpg")
print(a.shape)

(460, 816, 3)


1.  Resizes all images to **400×400** pixels before feeding them into the model.

2.  Loads images in batches of 32 (instead of all at once). This helps with memory efficiency and faster training.

3.  Used for integer-labeled classes (e.g., 0, 1, 2, 3 instead of [0,1,0,0]).

In [5]:
train = ImageDataGenerator(rescale=1/255)
train_dataset = train.flow_from_directory(
    r'/content/drive/MyDrive/Data_set/Data',
    target_size=(400, 400),
    batch_size=32,
    class_mode='sparse'
)

Found 1000 images belonging to 4 classes.


In [6]:
num_classes = len(train_dataset.class_indices)
print("Number of classes:", num_classes)

Number of classes: 4


In [7]:
train_dataset.class_indices

{'bump': 0, 'crack': 1, 'potholes': 2, 'road': 3}

This is a **Convolutional Neural Network (CNN)** built using TensorFlow and Keras, designed for image classification.


1.   **Conv2D(16, (3,3), activation='relu', input_shape=(400, 400, 3))**
Applies 16 filters (small 3×3 matrices) to detect patterns (edges, textures).
Uses ReLU activation to introduce non-linearity.
input_shape=(400, 400, 3) → Expects 400×400 RGB images (3 channels for color).

 **MaxPool2D(2,2)**
Reduces the spatial size by taking the maximum value in a 2×2 window.
This helps in reducing computation and extracting important features.
2.  **Conv2D(32, (3,3), activation='relu') → 32** filters to capture more complex features.

 **MaxPool2D(2,2) → Again** reduces spatial size.
3.  **Conv2D(64, (3,3), activation='relu') → 64** filters for deeper feature extraction.

 **MaxPool2D(2,2) → Reduces** feature map size further.

4. Converts the 2D feature maps into a 1D vector so it can be processed by dense (fully connected) layers.

5.  A **dense layer** with 512 neurons and **ReLU** activation.
 This acts as a hidden layer to learn complex patterns from features.
6. Output layer with 4 neurons (suggesting 4 classes in classification).
 Uses **Softmax activation**, which converts outputs into probabilities (summing to 1).

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(400, 400, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

**Adam** (Adaptive Moment Estimation) is an advanced gradient descent optimization algorithm.

It combines:
1.  **Momentum-based updates** (helps in faster convergence).
2.  **Adaptive learning rates** (adjusts learning rates for each parameter).

Adam is commonly used in deep learning because it balances speed and accuracy well.

**sparse_categorical_crossentropy**

   1.  This is used when labels are integer-encoded (e.g., 0, 1, 2, 3).
   2.  If labels were one-hot encoded (e.g., [0,1,0,0]), we would use categorical_crossentropy.

**Working**
1.  Measures the difference between predicted class probabilities and actual class labels.
2.  Encourages the model to assign higher probabilities to the correct class.

**Metrics**
1.  The model will track accuracy during training and evaluation.
  
  **Accuracy= Correct prediction / Total prediction**

2.  Useful for classification problems where classes are balanced.

In [10]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy
    metrics=['accuracy']
)

**Forward Pass:**
The model makes predictions on train_dataset.

**Loss Calculation:**
Compares predictions with actual labels using sparse_categorical_crossentropy.

**Backward Pass (Gradient Descent Update):**
The Adam optimizer adjusts weights to minimize loss.

**Repeat for All Samples:**
This happens for every batch in the dataset.

**End of One Epoch:**
Once all samples are processed, one epoch is completed.
Next Epoch:
The model learns further with updated weights.


In [11]:
model.fit(train_dataset,
          epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 236s 7s/step - accuracy: 0.5022 - loss: 3.4714
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 362ms/step - accuracy: 0.8764 - loss: 0.3804
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 365ms/step - accuracy: 0.9379 - loss: 0.1829
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 363ms/step - accuracy: 0.9696 - loss: 0.0925
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 362ms/step - accuracy: 0.9690 - loss: 0.0956


In [29]:
def get_road_condition(class_index):
  if class_index == 0:
    return 'bump'
  elif class_index == 1:
    return 'crack'
  elif class_index == 2:
    return 'potholes'
  elif class_index == 3:
    return 'road'
  else:
    return 'unknown'

predicted_class = predict_image("/content/drive/MyDrive/Data_set/Data/road/Augusta9c.jpg")
road_condition = get_road_condition(predicted_class)
print(f"The predicted road condition is: {road_condition}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
The predicted road condition is: road
